In [1]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
size, M, batch = 8, 2, 20

np.random.seed(0)
N = batch * size
X1 = np.random.normal(scale=[1, 0.8], size=[N, M]) + np.array([0, 1.5])
X2 = np.random.normal(scale=[1, 0.5], size=[N, M]) + np.array([1.5, 0])

AA = np.vstack([
    -1 * np.hstack([X1, np.ones([N, 1])]),
    np.hstack([X2, np.ones([N, 1])])
])

random_indexes = np.random.permutation(N).reshape([size, -1])
random_indexes = np.hstack([random_indexes, random_indexes + N])

AA = np.stack([AA[random_indexes[idx], :] for idx in range(size)])

In [3]:
def w_feasible(W):
    W = np.array(W)
    if W.ndim != 2 or (W.ndim == 2 and W.shape[0] != W.shape[1]):
        return False
    m, _ = W.shape
    return np.all(W == W.T) and \ 
        np.all(np.sum(W, axis=1) == np.ones(m)) == True and \
        np.all(np.sum(W, axis=0) == np.ones(m)) == True # symmetric and doubly stochastic matrix


def ring_matrix(n):
    w = np.eye(n) * 0.5
    for i in range(n):
        w[i, (i+1)%n] = 0.25
        w[i, (i-1)%n] = 0.25
    return w

In [4]:
W = ring_matrix(size)
w_feasible(W)

True

In [5]:
class GCon():
    def __init__(self, shape, W):
        self.x_last = np.zeros(shape)
        self.s_last = np.zeros(shape)
        self.W = W # TODO: check w feasiblity
        self.W2 = W.dot(W - np.eye(W.shape[0]))
    
    def __call__(self, x):
        s = self.W.dot(self.s_last) + self.W2.dot(self.x_last)
        self.s_last, self.x_last = s, x
        return s + W.dot(x)


In [6]:
MAX_ITER = 500 + 1
ABSTOL = 1e-4
RELTOL = 1e-2
rho, C = 1, 1

z, u, x = np.zeros([size, M+1]), np.zeros([size, M+1]), np.zeros([size, M+1])
x_con = GCon(x.shape, W)
u_con = GCon(x.shape, W)

STOP_FLAG, i = False, 0

while True:
    if STOP_FLAG or i >= MAX_ITER:
        break
    for wk in range(size):
        A = AA[wk]
        x_var = cp.Variable(M+1)
        cp.Problem(
            cp.Minimize(cp.sum(cp.pos(cp.matmul(A, x_var)+1)) + rho/2 * cp.sum_squares(x_var-z[wk]+u[wk]))
        ).solve(verbose=False, solver=cp.CVXOPT)
        x[wk] = x_var.value

    x_ave = x_con(x)
    z = rho/(1/C + size*rho) * size * (x_ave + u_con(u))
    u += x - z
    
    if i % 100 == 0:
        print(i, x_ave)
    
    i += 1

0 [[-0.66173666  1.60539364 -0.45211516]
 [-0.51528489  1.58112613 -0.67117216]
 [-0.50227951  1.38136738 -0.57590417]
 [-0.64498382  1.33692201 -0.44861301]
 [-0.77061423  1.48912667 -0.51532829]
 [-0.71575697  1.6040917  -0.6768552 ]
 [-0.60038342  1.48328885 -0.6978469 ]
 [-0.65898815  1.4370367  -0.44455414]]
100 [[-0.6897898   1.88286263 -0.73249575]
 [-0.68945605  1.88281248 -0.73283814]
 [-0.68891869  1.88261544 -0.73333932]
 [-0.68849172  1.88238504 -0.73370051]
 [-0.68843049  1.88225547 -0.73371852]
 [-0.68877168  1.88230404 -0.73338769]
 [-0.68931117  1.88250367 -0.73289431]
 [-0.6897307   1.88273564 -0.73252156]]
200 [[-0.67460616  1.87496501 -0.68666987]
 [-0.67474162  1.87470237 -0.68615602]
 [-0.67419984  1.87466565 -0.68564865]
 [-0.67327116  1.87489828 -0.68548043]
 [-0.67252647  1.87524807 -0.68573127]
 [-0.67242438  1.87549043 -0.68621919]
 [-0.67301306  1.87549402 -0.68668538]
 [-0.67390837  1.87528167 -0.68687952]]
300 [[-0.67489295  1.87646751 -0.68626354]
 [-0.675

In [9]:
ring_matrix(8)

array([[0.5 , 0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.25],
       [0.25, 0.5 , 0.25, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.25, 0.5 , 0.25, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.25, 0.5 , 0.25, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.25, 0.5 , 0.25, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.25, 0.5 , 0.25, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.5 , 0.25],
       [0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.5 ]])